In [4]:
from botasaurus_api import Api
import json
import csv

In [38]:
import os
import time
import pandas as pd 

In [5]:
api = Api()

# Experimental Cells from here till marker below, don't need to run

In [42]:
# Create an asynchronous task
data = {
    'queries': ['Peach Garden @ The Heeren'],
    'country': None,
    'business_type': '',
    'max_cities': None,
    'randomize_cities': True,
    'api_key': '',
    'enable_reviews_extraction': True,
    'max_reviews': 50,
    'reviews_sort': 'newest',
    'lang': None,
    'max_results': None,
    'coordinates': '',
    'zoom_level': 14,
}

In [43]:
task = api.create_sync_task(data, scraper_name='google_maps_scraper')

View create_sync_task response at: ./output/responses/create_sync_task.json


In [46]:
# Fetch the task results
results = api.get_task_results(task_id=task[0]['id'])

View get_task_results response at: ./output/responses/get_task_results.json


In [32]:
featured_reviews = results['results'][0]['featured_reviews']

In [47]:
detailed_reviews = results['results'][0]['detailed_reviews']

In [53]:
results['results'][0]

{'place_id': 'ChIJq-zcK98Z2jER-biFPzPW_hs',
 'name': 'Peach Garden @ The Heeren',
 'description': '*****',
 'is_spending_on_ads': '*****',
 'reviews': 913,
 'competitors': '*****',
 'website': '*****',
 'can_claim': '*****',
 'owner': '*****',
 'featured_image': '*****',
 'main_category': 'Chinese restaurant',
 'categories': ['Chinese restaurant', 'Dim sum restaurant'],
 'rating': 4.6,
 'workday_timing': '*****',
 'is_temporarily_closed': '*****',
 'is_permanently_closed': '*****',
 'closed_on': '*****',
 'phone': '*****',
 'address': '260 Orchard Rd, #05-02 The Heeren, Singapore 238855',
 'review_keywords': '*****',
 'link': 'https://www.google.com/maps/place/Peach+Garden+@+The+Heeren/@1.3027635,103.837358,17z/data=!3m1!4b1!4m6!3m5!1s0x31da19df2bdcecab:0x1bfed6333f85b8f9!8m2!3d1.3027635!4d103.837358!16s%2Fg%2F11v6k52bg7?authuser=0&entry=ttu&g_ep=EgoyMDI0MDgyMS4wIKXMDSoASAFQAw%3D%3D',
 'status': '*****',
 'price_range': '*****',
 'reviews_per_rating': '*****',
 'featured_question': '**

In [61]:
results['results'][0]['name'].replace(' ', '_')

'Peach_Garden_@_The_Heeren'

In [63]:
f"{results['results'][0]['main_category'].split()[0]}__{results['results'][0]['name'].replace(' ', '_')}"

'Chinese__Peach_Garden_@_The_Heeren'

In [106]:
results['results'][0]['rating']

4.5

In [38]:
json_data_path = 'output/responses/get_task_results.json'

# Load the JSON data
with open(json_data_path, 'r') as file:
    data = json.load(file)

# Define the CSV file path
csv_file_path = 'output/responses/detailed_reviews.csv'

# Open the CSV file for writing
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
    # Define the fieldnames for the CSV
    fieldnames = [
        'place_id', 'name', 'reviews', 'main_category', 'categories', 'rating', 'address', 'link', 'review_id', 'rating', 'review_text', 
        'published_at', 'published_at_date', 'review_likes_count', 'response_from_owner_text', 'response_from_owner_ago', 'response_from_owner_date',
        'total_number_of_reviews_by_reviewer', 'total_number_of_photos_by_reviewer', 'is_local_guide', 'review_translated_text', 'response_from_owner_translated_text'
    ]
    
    # Create a CSV DictWriter object
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    # Write the header row
    writer.writeheader()
    
    # Process each result in the JSON data
    for result in data['results']:
        place_id = result.get('place_id')
        name = result.get('name')
        reviews = result.get('reviews')
        main_category = result.get('main_category')
        rating = result.get('rating')
        address = result.get('address')
        link = result.get('link')

        # Process each featured review
        for review in result.get('detailed_reviews', []):
            # Create a dictionary for the current review
            row = {
                'place_id': place_id,
                'name': name,
                'reviews': reviews,
                'main_category': main_category,
                'rating': rating,
                'address': address,
                'link': link,
                'review_id': review.get('review_id', ''),
                'rating': review.get('rating', ''),
                'review_text': review.get('review_text', '').replace('\n', ' ').replace('\r', ''),
                'published_at': review.get('published_at', ''),
                'published_at_date': review.get('published_at_date', ''),
                'review_likes_count': review.get('review_likes_count', ''),
                'response_from_owner_text': review.get('response_from_owner_text', ''),
                'response_from_owner_ago': review.get('response_from_owner_ago', ''),
                'response_from_owner_date': review.get('response_from_owner_date', ''),
                'total_number_of_reviews_by_reviewer': review.get('total_number_of_reviews_by_reviewer', ''),
                'total_number_of_photos_by_reviewer': review.get('total_number_of_photos_by_reviewer', ''),
                'is_local_guide': review.get('is_local_guide', ''),
                'review_translated_text': review.get('review_translated_text', ''),
                'response_from_owner_translated_text': review.get('response_from_owner_translated_text', ''),
            }
            # Write the row to the CSV file
            writer.writerow(row)

print(f"Data has been written to {csv_file_path}")


Data has been written to output/responses/detailed_reviews.csv


In [35]:
len(pd.read_csv('data/restaurant_names.csv'))

4590

# Refactored Code, can run cells from here on

In [6]:
def extract_names(path):
    restaurants = []
    with open(path, newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
        for row in reader:
            restaurants.append(' '.join(row).rstrip(","))
    return restaurants

In [7]:
def scrape_restaurant(restaurant):
    data = {
        'queries': [restaurant],
        'country': None,
        'business_type': '',
        'max_cities': None,
        'randomize_cities': True,
        'api_key': '',
        'enable_reviews_extraction': True,
        'max_reviews': None,
        'reviews_sort': 'newest',
        'lang': None,
        'max_results': None,
        'coordinates': '',
        'zoom_level': 14,
    }
    task = api.create_sync_task(data, scraper_name='google_maps_scraper')
    results = api.get_task_results(task_id=task[0]['id'])
    return results

In [118]:
# OLD FUNCTION, WON'T BE USED BUT KEPT FOR REFERENCE

def write_to_csv(results):
    detailed_reviews = results['results'][0]['detailed_reviews']
    file_name = f"{results['results'][0]['name'].replace(' ', '_')}"
    json_data_path = 'output/responses/get_task_results.json'
    
    # Load the JSON data
    with open(json_data_path, 'r') as file:
        data = json.load(file)
    
    # Define the CSV file path
    csv_file_path = f'output/responses/{file_name}.csv'
    
    # Open the CSV file for writing
    with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
        # Define the fieldnames for the CSV
        fieldnames = [
            'place_id', 'name', 'reviews', 'main_category', 'categories', 'main_rating', 'address', 'link', 'review_id', 'rating', 'review_text', 
            'published_at', 'published_at_date', 'review_likes_count', 'response_from_owner_text', 'response_from_owner_ago', 'response_from_owner_date',
            'total_number_of_reviews_by_reviewer', 'total_number_of_photos_by_reviewer', 'is_local_guide', 'review_translated_text', 'response_from_owner_translated_text'
        ]
        
        # Create a CSV DictWriter object
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        # Write the header row
        writer.writeheader()
        
        # Process each result in the JSON data
        for result in data['results']:
            place_id = result.get('place_id')
            name = result.get('name')
            reviews = result.get('reviews')
            main_category = result.get('main_category')
            categories = result.get('categories')
            main_rating = result.get('rating')
            address = result.get('address')
            link = result.get('link')
    
            # Process each featured review
            for review in result.get('detailed_reviews', []):
                # Create a dictionary for the current review
                row = {
                    'place_id': place_id,
                    'name': name,
                    'reviews': reviews,
                    'main_category': main_category,
                    'categories': categories,
                    'main_rating': main_rating,
                    'address': address,
                    'link': link,
                    'review_id': review.get('review_id', ''),
                    'rating': review.get('rating', ''),
                    'review_text': review.get('review_text', '').replace('\n', ' ').replace('\r', ''),
                    'published_at': review.get('published_at', ''),
                    'published_at_date': review.get('published_at_date', ''),
                    'review_likes_count': review.get('review_likes_count', ''),
                    'response_from_owner_text': review.get('response_from_owner_text', ''),
                    'response_from_owner_ago': review.get('response_from_owner_ago', ''),
                    'response_from_owner_date': review.get('response_from_owner_date', ''),
                    'total_number_of_reviews_by_reviewer': review.get('total_number_of_reviews_by_reviewer', ''),
                    'total_number_of_photos_by_reviewer': review.get('total_number_of_photos_by_reviewer', ''),
                    'is_local_guide': review.get('is_local_guide', ''),
                    'review_translated_text': review.get('review_translated_text', ''),
                    'response_from_owner_translated_text': review.get('response_from_owner_translated_text', ''),
                }
                # Write the row to the CSV file
                writer.writerow(row)
    
    print(f"Data has been written to {csv_file_path}")

In [36]:
def write_to_csvs(results, folder_path):
    detailed_reviews = results['results'][0]['detailed_reviews']
    file_name = f"{results['results'][0]['name'].replace(' ', '_')}"
    json_data_path = 'output/responses/get_task_results.json'
    
    # Load the JSON data
    with open(json_data_path, 'r') as file:
        data = json.load(file)
    
    # Save restaurant details
    restaurant_file_path = f'output/responses/restaurant_dataset.csv'
    with open(restaurant_file_path, 'a', newline='', encoding='utf-8') as csvfile:        
        restaurant_file_fieldnames = [
            'place_id', 'name', 'reviews', 'main_category', 'categories', 'main_rating', 'address', 'link', 'review_photos'
        ]
        restaurant_writer = csv.DictWriter(csvfile, fieldnames=restaurant_file_fieldnames)

        # Write headers only for first run
        # restaurant_writer.writeheader()

        for result in data['results']:
            review_photos = []
            for review in result.get('featured_reviews', []):
                review_photos.extend(review.get('review_photos', ''))

            row = {
                'place_id': result.get('place_id'),
                'name': result.get('name'),
                'reviews': result.get('reviews'),
                'main_category': result.get('main_category'),
                'categories': result.get('categories'),
                'main_rating': result.get('rating'),
                'address': result.get('address'),
                'link': result.get('link'),
                'review_photos': review_photos
            }
            restaurant_writer.writerow(row)

    # Save reviews details
    reviews_file_path = f'{folder_path}/{file_name}.csv'

    with open(reviews_file_path, 'w', newline='', encoding='utf-8') as csvfile:        
        reviews_file_fieldnames = [
            'place_id', 'review_id', 'rating', 'review_text', 
            'published_at', 'published_at_date', 'review_likes_count', 'response_from_owner_text', 'response_from_owner_ago', 'response_from_owner_date',
            'total_number_of_reviews_by_reviewer', 'total_number_of_photos_by_reviewer', 'is_local_guide', 'review_translated_text', 'response_from_owner_translated_text'
        ]        
        reviews_writer = csv.DictWriter(csvfile, fieldnames=reviews_file_fieldnames)        
        reviews_writer.writeheader()
        for result in data['results']:
            # Process each featured review
            for review in result.get('detailed_reviews', []):
                # Create a dictionary for the current review
                row = {
                    'place_id': result.get('place_id'),
                    'review_id': review.get('review_id', ''),
                    'rating': review.get('rating', ''),
                    'review_text': review.get('review_text', '').replace('\n', ' ').replace('\r', ''),
                    'published_at': review.get('published_at', ''),
                    'published_at_date': review.get('published_at_date', ''),
                    'review_likes_count': review.get('review_likes_count', ''),
                    'response_from_owner_text': review.get('response_from_owner_text', ''),
                    'response_from_owner_ago': review.get('response_from_owner_ago', ''),
                    'response_from_owner_date': review.get('response_from_owner_date', ''),
                    'total_number_of_reviews_by_reviewer': review.get('total_number_of_reviews_by_reviewer', ''),
                    'total_number_of_photos_by_reviewer': review.get('total_number_of_photos_by_reviewer', ''),
                    'is_local_guide': review.get('is_local_guide', ''),
                    'review_translated_text': review.get('review_translated_text', ''),
                    'response_from_owner_translated_text': review.get('response_from_owner_translated_text', ''),
                }
                reviews_writer.writerow(row)
    
    print(f"Data has been written to {restaurant_file_path}")
    print(f"Data has been written to {reviews_file_path}")

# Run below cell to get the list of restaurant names

In [9]:
# Replace restaurant_list_path with the right file
restaurant_list_path = 'data/restaurant_names.csv'
restaurants = extract_names(restaurant_list_path)

In [10]:
restaurants

['(Fu Chan) Harummanis Muslim Food',
 '(Viet Taste) Vietnamese Cuisine @ Food Junction Great World City',
 '#01-22 Wufu Hakka Noodle',
 '#Foodcoholic',
 '#julkitchen',
 '116 PHỐ ĐÊM',
 '123 ZÔ - Ẩm Thực Việt',
 '123 Zô The Việt Kitchen',
 '126 Dim Sum @ Boon Keng',
 '13% Wine Bistro @Telok Ayer',
 '15 Stamford',
 '168 CMY Satay',
 '17 Turkish Doner Kebab and Catering',
 '1918 Heritage Bar at The Alkaff Mansion',
 '2 Béo Vietnamese Restaurant',
 '21 on Rajah',
 '22 Grams Café',
 '225SmokeHouse@Padang',
 '2612 Little Vietnam Cuisine',
 '27 Mezze Bar and Grill',
 '27 Mezze Bar and Grill',
 '27 Mezze Bar and Grill',
 '2am: dessertbar',
 '2it & Drink',
 '341 Beach Road Pho (Original)',
 '358 Xiang Wei Mala Hotpot',
 '5 Little Bears五隻小熊',
 '5 ON 25',
 '5 ON 25',
 '5 ON 25',
 '5 ON 25',
 '5-Star Dim Sum - Meiling Street',
 '51 Tras',
 '52 Coffeeshop',
 '55 Steamboat',
 '57˚ Mala Xiang Guo',
 '57˚ Mala Xiang Guo @Admiralty',
 '57˚ Mala Xiang Guo @Paya Lebar',
 '57° Mala Xiang Guo @Ang Mo Kio',

# Below cell starts the scraping at scale - read code first before running!

In [47]:
# Parameters to adjust
batch_size = 10
# Select which num_of_restaurants to uncomment
num_of_restaurants = 30  # to just scrape a portion
# num_of_restaurants = len(pd.read_csv('data/restaurant_names.csv')) + 1 # to scrape the full list of restaurants i.e. 4591

# Don't touch
num_of_batches = num_of_restaurants // batch_size
start_index = 0
stop_index = start_index + batch_size

# Extract by batch
for i in range(0, num_of_batches):
    
    # Create batch folder
    batch_folder_path = f'output/responses/batch_{i}'
    os.makedirs(batch_folder_path, exist_ok=True)

    # Loop scraping on specific indices of batch
    print(f"Now creating batch {i} from restaurants[{start_index}:{stop_index}]")
    for restaurant in restaurants[start_index:stop_index]:
        results = scrape_restaurant(restaurant)
        write_to_csvs(results, batch_folder_path)        

    # Increment start_index to prep for next loop
    start_index = start_index + batch_size
    stop_index = start_index + batch_size
    time.sleep(20)
    print("Waiting 20 seconds.")

Now creating batch 0 from restaurants[0:3]
View create_sync_task response at: ./output/responses/create_sync_task.json
View get_task_results response at: ./output/responses/get_task_results.json
Data has been written to output/responses/restaurant_dataset.csv
Data has been written to output/responses/batch_0/(Fu_Chan)_Harummanis_Muslim_Food.csv
View create_sync_task response at: ./output/responses/create_sync_task.json
View get_task_results response at: ./output/responses/get_task_results.json
Data has been written to output/responses/restaurant_dataset.csv
Data has been written to output/responses/batch_0/(Viet_Taste)_Vietnamese_Cuisine_@_Food_Junction_Great_World_City.csv
View create_sync_task response at: ./output/responses/create_sync_task.json
View get_task_results response at: ./output/responses/get_task_results.json
Data has been written to output/responses/restaurant_dataset.csv
Data has been written to output/responses/batch_0/#01-22_Wufu_Hakka_Noodle.csv
Waiting 20 seconds.
N

# Below cell will combine all the reviews in each batch folder into a single file 'reviews_dataset.csv'. Run this cell when the scraping is all done.

In [55]:
# Concatenate the reviews in each batch folder
root_folder_path = 'output/responses'
dataframes = []

# Loop through each item in the root directory
for item in os.listdir(root_folder_path):
    item_path = os.path.join(root_folder_path, item)
    
    # Check if the item is a directory (i.e., a batch folder)
    if os.path.isdir(item_path):
        
        # List all CSV files in the current batch folder
        csv_files = [f for f in os.listdir(item_path) if f.endswith('.csv')]
        
        # Loop through the list of CSV files and read them into DataFrames
        for file in csv_files:
            file_path = os.path.join(item_path, file)
            df = pd.read_csv(file_path)
            dataframes.append(df)

# Concatenate all DataFrames into a single DataFrame
reviews_df = pd.concat(dataframes, ignore_index=True)

# Save the combined DataFrame to a new CSV file
reviews_csv_path = 'output/responses/reviews_dataset.csv'
reviews_df.to_csv(reviews_csv_path, index=False)

print(f"Reviews CSV saved to {reviews_csv_path}")


Reviews CSV saved to output/responses/reviews_dataset.csv


In [57]:
reviews_df.shape

(1396, 15)